# Conservation law · Logistic invariant walkthrough

This notebook drives `code/conservation_law/qfum_validate.py` on a small
parameter set to illustrate the conserved quantity $Q$ and show where
figures/logs would be written when running the full validator.

In [ ]:
import sys
from pathlib import Path

def _ensure_repo_on_path():
    here = Path.cwd().resolve()
    for candidate in (here, *list(here.parents)[:4]):
        utils_py = candidate / 'notebooks' / 'utils.py'
        if utils_py.exists():
            path_str = str(candidate)
            if path_str not in sys.path:
                sys.path.insert(0, path_str)
            return candidate
    return here

_ensure_repo_on_path()


In [ ]:
from notebooks.utils import ensure_repo_path, allocate_artifacts, preview_json

repo_root = ensure_repo_path()
print(f"Repo root: {repo_root}")

In [ ]:
try:
    from code.conservation_law.qfum_validate import integrate_numeric, logistic_analytic, Q_invariant
except ModuleNotFoundError:
    from notebooks.utils import ensure_repo_path as _vdm_ensure_repo_path
    _vdm_ensure_repo_path()
    import importlib
    importlib.invalidate_caches()
    from code.conservation_law.qfum_validate import integrate_numeric, logistic_analytic, Q_invariant

import numpy as np

params = {"r": 0.15, "u": 0.25, "W0": 0.12, "T": 2.0, "dt": 0.01}
t, W_num = integrate_numeric(params["r"], params["u"], params["W0"], params["T"], params["dt"])
W_an = logistic_analytic(params["r"], params["u"], params["W0"], t)
Q = Q_invariant(params["r"], params["u"], W_num, t)
delta_Q = float(np.nanmax(np.abs(Q - Q[0])))
summary = {
    "delta_Q_max": delta_Q,
    "samples": int(t.size),
    "t_final": float(t[-1]),
}
print(preview_json(summary))


In [ ]:
artifacts = allocate_artifacts("conservation_law", "qfum_notebook_demo")
print(preview_json({name: str(path) for name, path in artifacts.items()}))

## Next steps

* Adjust `params` and re-run to explore convergence behaviour.
* For publication-ready figures/logs, run the CLI validator:
  ```bash
  python -m code.conservation_law.qfum_validate --r 0.15 --u 0.25 --W0 0.12 --T 40 --dt 0.001 --solver rk4
  ```
* Acceptance gates and full logging are described in the docstring of
  `qfum_validate.py` and mirrored in the repository tests.